# Basis-Set-Free VQEs with the Tequila - Madness interface

This tutorial covers the basic usage of the `tequila`-`madness` interface.  
It is currently brief and stenographic since the interface is still under development.  
The current version is however powerful enough to reproduce all results from our [article](https://arxiv.org/abs/2008.02819)!

One of `tequila`s primary aims is to simplify usage of many specialized algorithms and programs. The underlying technology should however be acknowledged, so if you are using the interface please acknowledge/cite the following articles:
- Our initial [article](https://arxiv.org/abs/2008.02819) describing the interface and introducint MRA-PNOs to quantum computing.
- The original [article](https://doi.org/10.1063/1.5141880) that describes the underlying MRA-PNO-MP2 implementation in `madness`.
- The `madness` overview [article](https://doi.org/10.1137/15M1026171).
- The `tequila` overview [article](https://arxiv.org/abs/2011.03057).
- In case you are using our automatically differentiable framework for unitary coupled-cluster, please also cite the [article](https://arxiv.org/abs/2011.05938) describing this simulation.
- Please also cite the corresponding articles of the quantum simulators and optimizers that you are using within `tequila`.

If you just want to play with the hydrogen molecule in a minimal basis-set-free representation without installing `madness`, there is a short tutorial on this as well!

## Installation of madness and depenencies

Currently the necessary version of `madness` is available over a fork under: https://github.com/kottmanj/madness . It is not yet merged into the main repository of madness, but will be at some point in the near future.  
You find installation instructions in the [README](https://github.com/kottmanj/madness) respectively github page of this fork. `madness` can be a beast to install, feel free to contact [me](mailto:jakob.kottmann@utoronto.ca) if you are running into trouble.

After you compiled madness from the fork (make sure the `tequila` branch was checked out - it is the default, but it doesn't hurt to check). There are two ways to let `tequila` know about is.
- export the path to the madness root (that is the directory where you compiled the code) as the environment variable 'MAD_ROOT_DIR'. E.g `export MAD_ROOT_DIR=/path/to/where/you/compiled/madness/`. You need to do this in your terminal (Linux or Mac) before you run your python script. `tequila` is then able to find the corresponding executable directly
- you provide the path to the `pno_integrals` executable yourself. E.g `tq.Molecule(..., executable="/path/to/madness/src/apps/pno/pno_integrals")`.

You can exploit the second option and provide the path to a customized script that will itself call `pno_integrals` in some way (this is for example an option if you are using windows and want to run madness over docker or some kind of virtual machine. See the above github fork of madness for more information on how to use it with docker).

In the following I will use the second option and provide the path to madness on my laptop. I'll leave that in, since that might clarify the usage a bit.

In [4]:
executable = "/home/jsk/devel/madness/build-opt/src/apps/pno/pno_integrals"

## Use Tequila to compute a QubitHamiltonian with madness

Here is how you can create a simple QubitHamiltonian over the interface.  
Checkout the main chemistry tutorial and see what you can further do with this (the usage is equivalent).  
- Choose different qubit encodints over the `transformation` keyword (the default is Jordan-Wigner - see the main chemistry tutorial)
- the `name` keyword is optional, the default is just `molecule`, it will mostly affect the output files names
- the `n_pno` keyword specifies how many pnos shall be computed and used. Note that the total number of orbitals is always `n_pno` plus all occupied Hartree-Fock orbitals.

In [7]:
import tequila as tq
geomstring = "He 0.0 0.0 0.0"
mol = tq.Molecule(geometry=geomstring, n_pno=1, executable=executable, name="helium_1pno")
H = mol.make_hamiltonian()

Starting madness calculation with executable:  /home/jsk/devel/madness/build-opt/src/apps/pno/pno_integrals
output redirected to helium_1pno_pno_integrals.out logfile
finished after 45.39036822319031s


In [8]:
# some information about the molecule
print(mol)

<class 'tequila.quantumchemistry.madness_interface.QuantumChemistryMadness'>
Qubit Encoding
transformation=<function jordan_wigner at 0x7f4b09b9ed08>

Parameters
basis_set       : custom          
geometry        : He 0.0 0.0 0.0  
description     :                 
multiplicity    : 1               
charge          : 0               
closed_shell    : True            
name            : helium_1pno     

MRA Orbitals    :
orbital 0, occupied reference orbital (0,), energy -0.917962 
orbital 1, pno from pair (0, 0), MP2 occ 0.00437667 

executable      : /home/jsk/devel/madness/build-opt/src/apps/pno/pno_integrals
htensor         : helium_1pno_htensor.npy
gtensor         : helium_1pno_gtensor.npy



## Read in already existing madness outputs

If you give `n_pno=None` tequila will try to read in the orbital data form madness.  
The following files are required: `name_htensor.npy` (or `molecule_htensor.bin`) `name_gtensor.npy` (or `molecule_gtensor.bin`) and `pnoinfo.txt`. Note that the `.bin` files should not be copied between different computers. `name` can be specific as keyword (default is just `molecule`).

In [10]:
mol2 = tq.Molecule(geometry=geomstring, n_pno=None, executable=executable, name="helium_1pno")
print(mol2)

<class 'tequila.quantumchemistry.madness_interface.QuantumChemistryMadness'>
Qubit Encoding
transformation=<function jordan_wigner at 0x7f4b09b9ed08>

Parameters
basis_set       : custom          
geometry        : He 0.0 0.0 0.0  
description     :                 
multiplicity    : 1               
charge          : 0               
closed_shell    : True            
name            : helium_1pno     

MRA Orbitals    :
orbital 0, occupied reference orbital (0,), energy -0.917962 
orbital 1, pno from pair (0, 0), MP2 occ 0.00437667 

executable      : /home/jsk/devel/madness/build-opt/src/apps/pno/pno_integrals
htensor         : helium_1pno_htensor.npy
gtensor         : helium_1pno_gtensor.npy



## Use the PNO-UpCCD ansatz
Check our [article](https://arxiv.org/abs/2008.02819) for more details on the ansatz. 
Note that the PNOs do not need to be directly determined over a basis-set-free approach for this ansatz, but this is currently the only implementation.  
Note, that the following four lines use all the technology mentioned in the first cell of this notebook. Please acknowledge it if you use it :-).
The used simulator is not specified hiere, so tequila will pick depending on what you have installed (you can see it in the printout).

In [17]:
import tequila as tq
geomstring = "He 0.0 0.0 0.0"
mol = tq.Molecule(geometry=geomstring, n_pno=1, executable=executable)
U = mol.make_pno_upccgsd_ansatz(generalized=False)
H = mol.make_hamiltonian()
E = tq.ExpectationValue(H=H, U=U)
result = tq.minimize(method="bfgs", objective=E, initial_values=0.0)

Starting madness calculation with executable:  /home/jsk/devel/madness/build-opt/src/apps/pno/pno_integrals
output redirected to molecule_pno_integrals.out logfile
finished after 9.489994764328003s
refs= [orbital 0, occupied reference orbital (0,), energy -0.917962 ]
Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
device          : None
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 6 expectationvalues

active variables : 3

E=-2.86152285  angles= {(0, ((0, 2), (1, 3)), None): 0.0, (0, (0, 2), None): 0.0, (0, (1, 3), None): 0.0}  samples= None
E=-2.85540235  angles= {(0, ((0, 2), (1, 3)), None): 0.2644369900226593, (0, (0, 2), None): 6.854534149169922e-06, (0, (1, 3), None): 6.854534149169922e-06}  samples= None
E=-2.87761682  angles= {(0, ((0, 2), (1, 3)), None): 0.12154174082914501, (0, (0, 2), None): 3.150512388571456e-06, (0, (1, 3),

## Customize the madness calculation

The underlying madness calculation can be customized by giving the corresponding sub-programs as keywords combined with dictionaries that hold the parameters.
To get an idea what is there: Check the output file `name_pno_integrals.out` generated by a madness calculation with default parameters and also the input file `input` writen by tequila. Here is some example how to customize it (here we change the polynomial order of the MRA which needs to be set in the scf calculation and increase the maximum rank of the PNOs which will result in 4 computed PNOs where the most important one is used for the QubitHamiltonian - this is just to illustrate the API, neither will have a notable effect on the Helium Hamiltonian with 1 PNO.). Note that the `scf` section is called `dft` in madness for historic reasons.

In [14]:
mol3 = tq.Molecule(geometry=geomstring, n_pno=1, executable=executable, name="helium_1pno_alt", dft={"k":9}, pno={"maxrank":4})


Starting madness calculation with executable:  /home/jsk/devel/madness/build-opt/src/apps/pno/pno_integrals
output redirected to helium_1pno_alt_pno_integrals.out logfile
finished after 37.68571734428406s


## Compute orbitals with frozen-core approximation
You can freeze occupied HF orbitals in the MP2-PNO calculation by setting the freeze keyword (e.g. `pno={"freeze":1}` or you just set `frozen_core=True` when intializing the Molecule.

In [15]:
mol = tq.Molecule(geometry="Be 0.0 0.0 0.0", n_pno=1, executable=executable, frozen_core=True)

Starting madness calculation with executable:  /home/jsk/devel/madness/build-opt/src/apps/pno/pno_integrals
output redirected to molecule_pno_integrals.out logfile
finished after 16.654462337493896s
